<a href="https://colab.research.google.com/github/SharovarovAleksandr/Academy/blob/main/Diplom2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [177]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from prophet import Prophet
import holidays
from holidays.financial import european_central_bank
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [214]:
def split_data(df, mark, start_test,end_test, predict_num):
    train=pd.DataFrame()
    test=pd.DataFrame()
    start_test=pd.to_datetime(start_test)
    end_test=pd.to_datetime(end_test)
    if start_test>=end_test :
        print('Error:  <end_test> go ahead of the <start_test>!')
        return None, None
    if (isinstance(df['time'][0], str)):
        df['time']=pd.to_datetime(df['time'])
    if end_test >  df['time'].iloc[-predict_num]:
        end_test=df['time'].iloc[-predict_num]
        print('Warning : <end_test> changed your date!')
    if start_test>=df['time'][len(df)-predict_num] or start_test<df['time'][0] :
        print('Error: <start_test> date of start is out of range DataFrame! ')
        return None, None
    if (predict_num>len(df)/2):
        print('Warning : <predict_num> too long period for pridiction')
    start_index=0
    end_index=0
    for i in range(len(df)):
        if start_test<=df['time'][i]:
            start_index=i
            break
    for i in range(len(df)):
        if end_test<=df['time'][i]:
            end_index=i
            break
    test['ds']=df['time'][end_index-predict_num+1:end_index+1]
    test['y']= df[mark][end_index-predict_num+1:end_index+1]
    # test=test.reset_index(drop=True)
    train['ds']=df['time'][start_index:end_index-predict_num+1]    
    train['y']=df[mark][start_index:end_index-predict_num+1] 
    train=train.reset_index(drop=True)
    return train, test

def holidays_tab(ticker, start_test, lower_window, upper_window): 
    import holidays
    end=(pd.to_datetime(df['time'][len(df)-1])).year
    start=(pd.to_datetime(start_test)).year
    list_year=[]
    for i in range(start,end+1):
      list_year.append(i)
    if ticker=='CA':
        canada_holidays = holidays.Canada(years=list_year, observed=True)
        df_holidays_ca = pd.DataFrame(canada_holidays.items(), columns=['Date', 'Holiday'])
        holidays_res = pd.DataFrame({'holiday': 'Canada holidays',
                                    'ds': pd.to_datetime(df_holidays_ca['Date']),
                                    'lower_window': lower_window,
                                    'upper_window': upper_window })

    elif ticker=='US':
        us_holidays = holidays.USA(years=list_year, observed=True)
        df_holidays_us = pd.DataFrame(us_holidays.items(), columns=['Date', 'Holiday'])
        holidays_res = pd.DataFrame({'holiday': 'USA holidays',
                                    'ds': pd.to_datetime(df_holidays_us['Date']),
                                    'lower_window': lower_window,
                                    'upper_window': upper_window })
    elif ticker=='GB':
        gb_holidays=holidays.GB(years=list_year, observed=True)
        df_holidays_gb = pd.DataFrame(gb_holidays.items(), columns=['Date', 'Holiday'])
        holidays_res = pd.DataFrame({'holiday': 'Great Britan holidays',
                                    'ds': pd.to_datetime(df_holidays_gb['Date']),
                                    'lower_window': lower_window,
                                    'upper_window': upper_window })

    elif ticker=='ECB':
        eur_holidays=holidays.ECB(years=list_year, observed=True)
        df_holidays_ecb = pd.DataFrame(eur_holidays.items(), columns=['Date', 'Holiday'])
        holidays_res = pd.DataFrame({'holiday': 'European Central Bank holidays',
                                    'ds': pd.to_datetime(df_holidays_ecb['Date']),
                                    'lower_window': lower_window,
                                    'upper_window': upper_window })
    elif ticker=='NYSE':
        us_holidays_bank=holidays.NYSE(years=list_year, observed=True)
        df_holidays_nyse = pd.DataFrame(us_holidays_bank.items(), columns=['Date', 'Holiday'])
        holidays_res = pd.DataFrame({'holiday': 'NYSE holidays',
                                    'ds': pd.to_datetime(df_holidays_nyse['Date']),
                                    'lower_window': lower_window,
                                    'upper_window': upper_window })
    else :
      print('Sorry, I don`t now holidays ',ticker)
      return None
    return holidays_res

def concat_holiday(*args):
    return pd.concat((args))

def MovingAvarage(df, mark, window=15):
    rolling_mean = df.loc[:, ['time', mark]].copy()
    rolling_mean[mark] = rolling_mean[mark].rolling(window=window).mean()
    rolling_mean=rolling_mean[window:].reset_index(drop=True)
    return rolling_mean  

def all_day_holiday(df, mark, start_test, end_test, window=True, lower_window=0, upper_window=0 ):
    start=pd.to_datetime(start_test)
    end=pd.to_datetime(end_test)
    if start_test>=end_test :
        print('Error:  <end_test> go ahead of the <start_test>!')
        return None, None
    if (isinstance(df['time'][0], str)):
        df['time']=pd.to_datetime(df['time'])
    delta=end-start
    num_date=delta.days+1
    for i in range(len(df)):
        if start<=df['time'][i]:
            start_index=i
            break
    for i in range(len(df)):
        if end<=df['time'][i]:
            end_index=i
            break
    nm=num_date-(end_index-start_index)-1
    result=pd.DataFrame(columns=['holiday', 'ds'],index=pd.RangeIndex(nm))
    j=0
    k=0
    for i in range(num_date):
        cur_date=start+timedelta(days=i)
        if cur_date == df['time'][start_index+j]:
            j+=1
        else:
            result['ds'][k]=cur_date.date()
            k+=1
    result['holiday']='All_day_holiday'
    if window:
        result['lower_window']=lower_window
        result['upper_window']=upper_window
    return result

def ideal_absolut(df, mark, start, end):
    start=pd.to_datetime(start_test)
    end=pd.to_datetime(end_test)
    if start_test>=end_test :
        print('Error:  <end_test> go ahead of the <start_test>!')
        return None, None
    if (isinstance(df['time'][0], str)):
        df['time']=pd.to_datetime(df['time'])
    start_index=0
    end_index=0
    for i in range(len(df)):
        if start<=df['time'][i]:
            start_index=i
            break
    for i in range(len(df)):
        if end<=df['time'][i]:
            end_index=i
            break
    num_date=end_index-start_index+1
    result=pd.DataFrame(columns=['ds', 'y'], index=pd.RangeIndex(num_date))
    past_date = end - timedelta(days=num_date-1)
    for i in range(num_date+1):
        result['ds'][i]=(past_date+timedelta(days=i)).date()
    result['y']=df[mark][start_index:end_index+1].values
    return result 
    
def split_ds(df, predict_num):
    train=df[:-predict_num]
    test=df[-predict_num:]
    # test=test.reset_index(drop=True)
    return train, test



In [215]:
df=pd.read_csv('/content/drive/MyDrive/Python/Data/GBPCAD_D1.csv', delimiter=';')
start_test=df['time'][0]
end_test=df['time'][len(df)-1]


In [287]:
mark='high'
start_test = '2021-01-01'
# end_test = '2023-03-15'
# df[mark]=df[mark]*1000
predict_num = 3
df1=ideal_absolut(df,mark,start_test, end_test)
train,test = split_ds(df1, predict_num)
# train,test = split_data(df, mark, start_test,end_test, predict_num)
# test
# holiday=all_day_holiday(df, mark, start_test, end_test, window=True, lower_window=-1, upper_window=1 )

In [293]:
model = Prophet(holidays=None, 
                holidays_prior_scale=10, 
                changepoint_range=0.8, 
                changepoint_prior_scale=0.5,
                yearly_seasonality = 'auto',
                weekly_seasonality = 1,
                daily_seasonality = 1,
                growth = 'linear', 
                n_changepoints = 25,
                changepoints = None,
                seasonality_mode = 'additive', 
                seasonality_prior_scale = 10, 
                mcmc_samples = 0, 
                interval_width = 0.08, 
                uncertainty_samples = 1000,
                stan_backend = None)
                
# model.add_regressor(regr)
model.fit(train)
future = model.make_future_dataframe(periods=predict_num)
forecast = model.predict(future)
error=mean_absolute_error(test['y'],forecast['yhat'][-predict_num:])
print(error)
rez=pd.concat((test,forecast['yhat'][-predict_num:],(test['y'][:]-forecast['yhat'][-predict_num:])*100000), axis=1)
print(rez)


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5tobxlmp/t2kklzd4.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5tobxlmp/srdmcy3o.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.9/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=54186', 'data', 'file=/tmp/tmp5tobxlmp/t2kklzd4.json', 'init=/tmp/tmp5tobxlmp/srdmcy3o.json', 'output', 'file=/tmp/tmp5tobxlmp/prophet_modelfaguhpzb/prophet_model-20230323161811.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:18:11 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:18:11 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


0.009184442894420553
             ds        y      yhat            0
571  2023-03-15  1.66624  1.677165 -1092.488842
572  2023-03-16  1.66705  1.678004 -1095.383144
573  2023-03-17  1.67386  1.679535  -567.460882


In [278]:
future = model.make_future_dataframe(periods=predict_num)
# future['hl_predict']=df['hl_predict']
# future.tail(5)


In [279]:
forecast = model.predict(future)
forecast.tail(4)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
830,2023-03-14,1.620704,1.637174,1.639696,1.620704,1.620704,0.017345,0.017345,0.017345,0.040669,...,0.000507,0.000507,0.000507,-0.023831,-0.023831,-0.023831,0.0,0.0,0.0,1.638049
831,2023-03-15,1.621306,1.636121,1.638880,1.621306,1.621306,0.016380,0.016380,0.016380,0.040669,...,-0.000531,-0.000531,-0.000531,-0.023758,-0.023758,-0.023758,0.0,0.0,0.0,1.637686
832,2023-03-16,1.621908,1.636364,1.639129,1.621908,1.621908,0.016479,0.016479,0.016479,0.040669,...,-0.000610,-0.000610,-0.000610,-0.023580,-0.023580,-0.023580,0.0,0.0,0.0,1.638388
833,2023-03-17,1.622511,1.639311,1.641667,1.622511,1.622511,0.016915,0.016915,0.016915,0.040669,...,-0.000439,-0.000439,-0.000439,-0.023315,-0.023315,-0.023315,0.0,0.0,0.0,1.639425


In [280]:
error=mean_absolute_error(test['y'],forecast['yhat'][-predict_num:])
print(error)
rez=pd.concat((test,forecast['yhat'][-predict_num:],(test['y'][:]-forecast['yhat'][-predict_num:])*100000), axis=1)
print(rez)
# test, forecast['yhat'][-predict_num:]
# print(forecast['yhat'][-predict_num:])


0.030550172847953805
             ds        y      yhat            0
831  2023-03-15  1.66624  1.637686  2855.355049
832  2023-03-16  1.66705  1.638388  2866.231219
833  2023-03-17  1.67386  1.639425  3443.465586


In [294]:
fig = make_subplots()
fig.update_layout(height=800, width=1400)
fig.add_trace(go.Line(x=df1['ds'], y=df1['y']))
fig.add_trace(go.Line(x=forecast['ds'], y=forecast['yhat']))
fig.show()

/usr/local/lib/python3.9/dist-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


